In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import ipywidgets as widgets
from IPython.display import display

In [2]:
def select_rows(df, cols = 5):
    checkboxes = []
    for i, row in df.iterrows():
        checkbox = widgets.Checkbox(
            value = False,
            description = f'{row["Food"]}',
        )
        checkboxes.append(checkbox)

    grid = widgets.GridBox(checkboxes, layout = widgets.Layout(
        grid_template_columns = f'repeat({cols}, 150px)',
        grid_gap='10px 10px'
    ))

    def on_button_click(b):
        selected_rows = [i for i, checkbox in enumerate(checkboxes) if checkbox.value]
        selected_df = df.iloc[selected_rows]
        display(selected_df)
        global data
        data = selected_df

    button = widgets.Button(description = "Submit")
    button.on_click(on_button_click)

    display(grid, button)


def calculate_bmr(gender, weight, height, age):
    """Calculate Basal Metabolic Rate using the Mifflin-St Jeor Equation."""
    if gender.lower() == "m":
        return 10 * weight + 6.25 * height - 5 * age + 5
    elif gender.lower() == "f":
        return 10 * weight + 6.25 * height - 5 * age - 161


def calculate_tdee(bmr, activity_multiplier):
    """Calculate Total Daily Energy Expenditure."""
    return int(bmr * activity_multiplier)


def calculate_nutrients(daily_caloric_intake, goal):
    """Calculate daily intake of protein, carbohydrates, and fats based on the goal."""
    if goal == "cutting":
        protein = int(0.40 * daily_caloric_intake / 4)
        carbohydrates = int(0.40 * daily_caloric_intake / 4)
        fats = int(0.20 * daily_caloric_intake / 9)
    elif goal == "bulking":
        protein = int(0.25 * daily_caloric_intake / 4)
        carbohydrates = int(0.55 * daily_caloric_intake / 4)
        fats = int(0.20 * daily_caloric_intake / 9)
    elif goal == "maintaining":
        protein = int(0.30 * daily_caloric_intake / 4)
        carbohydrates = int(0.45 * daily_caloric_intake / 4)
        fats = int(0.25 * daily_caloric_intake / 9)
    fibre = int(0.014 * daily_caloric_intake)
    saturated_fats = int(0.10 * daily_caloric_intake / 9)
    return protein, carbohydrates, fats, fibre, saturated_fats


def input_validation(prompt, data_type):
    while True:
        try:
            return data_type(input(prompt))
        except ValueError:
            print(f"Invalid input. Please enter a valid {data_type.__name__}.")


def format_weight(weight):
    if weight < 1000:
        return f"{weight} g"
    else:
        return f"{weight / 1000} kg"

In [9]:
vitaminsRDA = pd.read_csv("nutrient-databases/vitamins-RDAs.csv")
vitaminsUL = pd.read_csv("nutrient-databases/vitamins-ULs.csv")
elementsRDA = pd.read_csv("nutrient-databases/elements-RDAs.csv")
elementsUL = pd.read_csv("nutrient-databases/elements-ULs.csv")

In [4]:
gender = input("Enter your gender (m/f): ")
weight = input_validation("Enter your weight (in kg): ", int)
height = input_validation("Enter your height (in cm): ", int)
age = input_validation("Enter your age (in years): ", int)
activity_multiplier = input_validation("Enter your activity multiplier: ", float)
goal = input("Enter your goal (cutting/bulking/maintaining): ")

In [ ]:
bmr = calculate_bmr(gender, weight, height, age)
tdee = calculate_tdee(bmr, activity_multiplier)
daily_caloric_intake = {
    "cutting": 0.75 * tdee,
    "bulking": 1.10 * tdee,
    "maintaining": tdee,
}.get(goal, tdee)

protein, carbohydrates, fats, fibre, saturated_fats = calculate_nutrients(
    int(daily_caloric_intake), goal
)

print(f"BMR: {bmr}")
print(f"TDEE: {tdee}")
print(f"Daily Caloric Intake: {int(daily_caloric_intake)}")
print(f"Protein (g): {protein}")
print(f"Carbohydrates (g): {carbohydrates}")
print(f"Fats (g): {fats}")
print(f"Fibre (g): {fibre}")
print(f"Saturated Fats (g): {saturated_fats}")

In [ ]:
food_items = pd.read_csv("sample.csv")
select_rows(food_items, cols = 5)

In [ ]:
c = data["Price"].values

A = [
    -data["Protein (g)"].values,
    data["Protein (g)"].values,
    -data["Carbohydrates (g)"].values,
    data["Carbohydrates (g)"].values,
    -data["Fats (g)"].values,
    data["Fats (g)"].values,
    -data["Fibre (g)"].values,
    data["Fibre (g)"].values,
    data["Saturated Fats (g)"].values,
]

b = [
    -protein,
    protein * 1.01,
    -carbohydrates,
    carbohydrates * 1.01,
    -fats,
    fats * 1.01,
    -fibre,
    fibre * 1.01,
    saturated_fats,
]

bounds = [(0, None) for _ in range(len(data))]

In [ ]:
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

if result.success:
    print("Optimization succeeded.")
else:
    print("No solution found.")

In [ ]:
if result.success:
    servings = np.round(result.x, 1)
    quantity = np.round(servings * data["Serving (g)"], 1)
    total_cost_per_item = np.round(servings * data["Price"], 1)
    total_protein = np.round(servings * data["Protein (g)"], 1)
    total_carbohydrates = np.round(servings * data["Carbohydrates (g)"], 1)
    total_fats = np.round(servings * data["Fats (g)"], 1)
    total_fibre = np.round(servings * data["Fibre (g)"], 1)
    total_saturated_fats = np.round(servings * data["Saturated Fats (g)"], 1)
    
    result_df = pd.DataFrame(
        {
            "Food Item": data["Food"],
            "Optimal Servings": servings,
            "Optimal Quantity": quantity,
            "Total Cost": total_cost_per_item,
            "Total Protein (g)": total_protein,
            "Total Carbohydrates (g)": total_carbohydrates,
            "Total Fats (g)": total_fats,
            "Total Fibre (g)": total_fibre,
            "Total Saturated Fats (g)": total_saturated_fats,
        }
    )
    result_df['Optimal Quantity'] = result_df['Optimal Quantity'].apply(format_weight)
    print(result_df[result_df.iloc[:, 1:].ne(0).any(axis=1)])

In [ ]:
if result.success:
    total_summary = result_df.drop(columns=["Food Item", "Optimal Servings", "Optimal Quantity"]).sum()
    total_summary["Total Calories"] = (
        (total_summary["Total Protein (g)"] * 4)
        + (total_summary["Total Carbohydrates (g)"] * 4)
        + (total_summary["Total Fats (g)"] * 9)
    )
    print("\n Daily Summary:")
    print(total_summary)

In [ ]:
if result.success:
    weekly_cost = result_df["Total Cost"].sum() * 7
    print(f"\nWeekly Cost: {round(weekly_cost, 1)}")